In [ ]:
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
import rasterio
from rasterio.plot import show
from rasterio.merge import merge

from os.path import join
import glob
from PIL import Image
import PIL

PIL.Image.MAX_IMAGE_PIXELS = 933120000

In [ ]:
img_dir = r'D:\python\uni_work\ML_project\data_raw\canterbury-03m-rural-aerial-photos-2015-2016'
shp_dir = r'D:\python\uni_work\ML_project\data_raw\nz-building-outlines'

shapefile = join(shp_dir + r'/nz-building-outlines.shp')
ims = glob.glob(img_dir + '/*.tif')

In [ ]:
shp = gpd.read_file(shapefile)
savedir = r'D:\python\uni_work\ML_project\input\mask'

plt.rcParams['axes.xmargin'] = 0
plt.rcParams['axes.ymargin'] = 0

out_shape = (19879, 24579)

fig, ax = plt.subplots(figsize=(2.4579, 1.9879), frameon=False, dpi=1000)
fig.patch.set_facecolor('black')
shp.plot(ax=ax, figsize=(20,18), facecolor="None", color='white', aspect='auto')
plt.axis('off')
# %config InlineBackend.print_figure_kwargs = {'bbox_inches':None}
fig = plt.gcf()
DPI = fig.get_dpi()
fig.set_size_inches(24579.0/float(DPI),19879.0/float(DPI))
plt.gca().set_position([0, 0, 1, 1])
plt.axis('tight')

plt.show()
# plt.savefig(savedir + '\\rolleston_mask2.tif', dpi=1000)

In [ ]:
mosaic_in = []

for im in ims:
    tar_im = rasterio.open(im) 
    mosaic_in.append(tar_im)
    
mosaic, out_trans = merge(mosaic_in)

In [ ]:
from PIL import Image
from itertools import product
import os

def tile(filename, dir_in, dir_out, d):
    name, ext = os.path.splitext(filename)
    img = Image.open(os.path.join(dir_in, filename))
    w, h = img.size
    
    grid = product(range(0, h-h%d, d), range(0, w-w%d, d))
    for i, j in grid:
        box = (j, i, j+d, i+d)
        out = os.path.join(dir_out, f'{name}_{i}_{j}{ext}')
        img.crop(box).save(out)
        
tile('rolleston_mask2.tif', r'D:\python\uni_work\ML_project\input\mask', r'D:\python\uni_work\ML_project\input\training\mask', 80)
# tile(r'mosaic.tif', r'D:\python\uni_work\ML_project\input\mosaic', r'D:\python\uni_work\ML_project\input\training\raw', 80)

In [ ]:
import earthpy as et
import earthpy.spatial as es
import earthpy.plot as ep
from rasterio.plot import plotting_extent
savedir = r'D:\python\uni_work\ML_project\input\mask'
inpath = r'D:\python\uni_work\ML_project\input\mosaic\rolleston_black.jpg'

In [ ]:
backdrop = rasterio.open(r'D:\python\uni_work\ML_project\input\mosaic\mosaic.tif')
extent = plotting_extent(backdrop)

plt.rcParams['axes.xmargin'] = 0
plt.rcParams['axes.ymargin'] = 0

fig, ax = plt.subplots(figsize=(51.2, 51.2), frameon=False, dpi=1000)

ep.plot_rgb(backdrop.read(),
            ax=ax,
            extent=extent)

plt.axis('off')
ax.cla()
fig.patch.set_facecolor('black')
ax.set_axis_off()
shp.plot(ax=ax, aspect='auto', color='white')

plt.gca().set_position([0, 0, 1, 1])
plt.axis('tight')

plt.savefig(savedir + '\\rolleston_mask.tif', dpi=1000)

In [ ]:
backdrop = rasterio.open(r'D:\python\uni_work\ML_project\input\mosaic\mosaic.tif')
extent = plotting_extent(backdrop)

plt.rcParams['axes.xmargin'] = 0
plt.rcParams['axes.ymargin'] = 0

fig, ax = plt.subplots(figsize=(51.2, 51.2), frameon=False, dpi=1000)

ep.plot_rgb(backdrop.read(),
            ax=ax,
            extent=extent)

plt.axis('off')

plt.gca().set_position([0, 0, 1, 1])
plt.axis('tight')

plt.savefig(savedir + '\\rolleston_mosiac.tif', dpi=1000)

In [ ]:
# tile(filename, dir_in, dir_out, d)
tile('rolleston_mask.tif', r'D:\python\uni_work\ML_project\input\mask', r'D:\python\uni_work\ML_project\input\mask\mask', 512)
tile(r'mosaic.tif', r'D:\python\uni_work\ML_project\input\mask', r'D:\python\uni_work\ML_project\input\mask\truth', 512)